In [1]:
# -*- coding: utf-8 -*-
"""
Created on Sat Feb  9 01:47:27 2019

@author: avaca
"""
import pandas as pd
import numpy as np
import pickle
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.metrics import median_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer

In [2]:
data = pd.read_csv('transformados04.csv')

In [3]:
data['exit_rate_page_views'] = data['GA_exit_rate']*data['GA_page_views']
data['views_precio_metro'] = data['GA_page_views'] * data['precio_metros']
target = data['TARGET']

In [4]:
Xt = data.drop(['TARGET'], axis = 1)

yt = data['TARGET']

In [5]:
X_train, X_test, y_train, y_test = train_test_split(Xt, yt,
                                                    test_size = 0.1,
                                                    random_state = 69)

In [22]:
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

# Parámetros

In [30]:
# Número de árboles en el random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 1100, num = 100)]

# Número de features a considerar en cada split
max_features = ['sqrt'] # auto es peor

# Máximo número de niveles en el árbol
max_depth = [1]  # None, 2 y 3 son peores

# Número mínimo de samples para hacer un split en un nodo
min_samples_split = [2, 5, 10]

# Número mínimo de samples para cada hoja
min_samples_leaf = [1, 2, 4]

# Método de selección para cada árbol de entrenamiento
bootstrap = [True, False]

In [31]:
param_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [21]:
scorer = make_scorer(median_absolute_error)

# RandomForest

In [36]:
# rf = GridSearchCV(RandomForestRegressor(criterion="mse"),
#                   param_grid,
#                   cv = 5,
#                   scoring = scorer,
#                   n_jobs = -1,
#                   verbose = 1)

In [26]:
n_iteration = 1500

In [32]:
rf = RandomizedSearchCV(RandomForestRegressor(criterion="mse"),
                        param_grid,
                        cv = 5,
                        n_iter=n_iteration,
                        scoring = scorer,
                        n_jobs = -1,
                        verbose = 1)

In [34]:
rf.fit(X_train, y_train)

pred_rf = np.exp(rf.predict(X_test)) - 1
pred_rf = pred_rf.reshape(-1, 1)

df = pd.DataFrame(np.exp(y_test)-1 , index = y_test.index)
df.columns = ['TARGET']
df['pred_rf'] = pred_rf

median_absolute_error(df['TARGET'],df['pred_rf'])

Fitting 5 folds for each of 1500 candidates, totalling 7500 fits


KeyboardInterrupt: 

In [29]:
report(rf.cv_results_, n_top=20)

Model with rank: 1
Mean validation score: 0.552 (std: 0.014)
Parameters: {'n_estimators': 1054, 'min_samples_split': 5, 'min_samples_leaf': 4, 'max_features': 'sqrt', 'max_depth': 1, 'bootstrap': True}

Model with rank: 2
Mean validation score: 0.552 (std: 0.014)
Parameters: {'n_estimators': 545, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'max_depth': 1, 'bootstrap': True}

Model with rank: 3
Mean validation score: 0.552 (std: 0.014)
Parameters: {'n_estimators': 763, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'max_depth': 1, 'bootstrap': True}

Model with rank: 4
Mean validation score: 0.551 (std: 0.014)
Parameters: {'n_estimators': 254, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 1, 'bootstrap': False}

Model with rank: 5
Mean validation score: 0.551 (std: 0.013)
Parameters: {'n_estimators': 363, 'min_samples_split': 5, 'min_samples_leaf': 4, 'max_features': 'sqrt', 'max_depth': 1, 'bootstr

In [33]:
filename = 'random_forest_randomsearchcv3.sav'
pickle.dump(rf.best_estimator_, open(filename, 'wb'))

AttributeError: 'RandomizedSearchCV' object has no attribute 'best_estimator_'

# ExtraTreesRegressor

In [39]:
etr = GridSearchCV(ExtraTreesRegressor(criterion="mse"),
                   param_grid,
                   cv = 5,
                   scoring = scorer,
                   n_jobs = -1,
                   verbose = 1)

In [40]:
etr.fit(X_train, y_train)

pred_etr = np.exp(etr.predict(X_test)) - 1
pred_etr = pred_etr.reshape(-1, 1)

df['pred_etr'] = pred_etr

median_absolute_error(df['TARGET'],df['pred_etr'])

Fitting 5 folds for each of 400 candidates, totalling 2000 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  9.1min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed: 22.0min
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed: 39.2min
[Parallel(n_jobs=-1)]: Done 1242 tasks      | elapsed: 67.4min
[Parallel(n_jobs=-1)]: Done 1792 tasks      | elapsed: 108.3min
[Parallel(n_jobs=-1)]: Done 2000 out of 2000 | elapsed: 125.8min finished


23.623642862137796

In [41]:
# filename = 'extra_trees_gridsearchcv1.sav'
# pickle.dump(etr.best_estimator_, open(filename, 'wb'))